# Data reduction workflow for files generated on Larmor instrument

Some addhoc approximations will need to be done in order to satisfy SANS2D workflow, e.g direct measurments, tramsmission fraction. 

In [ ]:
import matplotlib.pyplot as plt
import scipp as sc
from ess import loki, sans
import scippneutron as scn
import plopp as pp
%matplotlib widget

## Setting up scicat connection

In [ ]:
#TODO: Placeholder for now

# import scitacean
# scitacean.__version__


# client = FakeClient(file_transfer=FakeFileTransfer(source_folder='/'))

# direct_beam_uploaded = get_direct_beam(client)
# sample_uploaded = get_sample(client, direct_beam_uploaded)

# sample_remote = client.get_dataset(sample_uploaded.pid)
# sample_local = client.download_files(sample_remote, target=".")


## Define reduction workflow parameters

We define here whether to include the effects of gravity,
as well as common time-of-flight, wavelength and $Q$ bins for all the measurements.

We also define a range of wavelengths for the monitors that are considered to not be part of the background.

In [ ]:
# Include effects of gravity?
gravity = True

#TODO: Using same wavelengths bins as direct beam is defined for but it may be different range (0.9-13.5)
wavelength_bins = sc.linspace(dim='wavelength', start=2.3, stop=11.9, num=110,
                              unit='angstrom')

q_bins = sc.linspace(dim='Q', start=0.008, stop=0.6, num=55, unit='1/angstrom')


## Loading data files

We load the following files:

- The direct beam function for the main detector (gives detector efficiency as a function of wavelength)
- The sample measurement
- The direct measurement: this is the run with the empty sample holder/cuvette
- the background measurement: this is the run with only the solvent which the sample is placed in

In [ ]:
#Loading Mantid
# filename = '60393-mantid-add.nxs'
# path = '/Users/wojciechpotrzebowski/SCIPP_SPACE/direct_beam_iterations_2023/'
# mantid_sample = scn.load(f'{path}/{filename}')
#mantid_sample

In [ ]:
#ds_id = sc.Dataset()

sample_run_number = 60395
sample_transmission_run_number = 60394
background_run_number = 60393
background_transmission_run_number = 60392
empty_beam_run_number = 60392

path = '/Users/wojciechpotrzebowski/SCIPP_SPACE/loki_detector_test/2022-06-24_calibrated_nexus_files/'
#direct_beam = loki.io.load_rkh_wav(f'{path}/DirectBeam_20feb_full_v3.dat')
direct_beam = scn.load(f'{path}/flat_g4_final_15Jun23_10-17.nxs')

sample = scn.load_nexus(data_file=f'{path}/{sample_run_number}-2022-02-28_2215.nxs')
sample = sample.rename(detector_id = 'spectrum')
#Mantid samples don't have montors, so we will need to pass them independently 
#sample = scn.load(f'{path}/{sample_run_number}-mantid-add.nxs')


sample_trans = scn.load_nexus(data_file=f'{path}/{sample_transmission_run_number}-2022-02-28_2215.nxs')
sample_trans = sample_trans.rename(detector_id = 'spectrum')

#TODO: How about direct?
#ds['direct'] = scn.load(filename=f'{path}/LARMOR000{background_run_number}.nxs')
empty_beam = scn.load_nexus(data_file=f'{path}/{empty_beam_run_number}-2022-02-28_2215.nxs')
empty_beam = empty_beam.rename(detector_id = 'spectrum')

background = scn.load_nexus(data_file=f'{path}/{background_run_number}-2022-02-28_2215.nxs')
background = background.rename(detector_id = 'spectrum')
#background = scn.load(f'{path}/{background_run_number}-mantid-add.nxs')

background_trans = scn.load_nexus(data_file=f'{path}/{background_transmission_run_number}-2022-02-28_2215.nxs')
background_trans = background_trans.rename(detector_id = 'spectrum')

ds = {'sample': sample, 'sample_trans': sample_trans, 'empty_beam': empty_beam, 
      'background': background, 'background_trans': background_trans}




## Apply corrections to pixel positions

We apply some corrections to the detector pixel and monitor positions,
as the geometry stored in the file is inaccurate.

In [ ]:
#sample_position 0, 0, 0 wrt to source?
# Now shift pixels positions to get the correct beam center
for da in ds.values():
    da.coords["sample_position"] = sc.vector(value=[0.0, 0.0, 0.0], unit='m')
    #sample_pos_z_offset = 0.1 * sc.units.m
    #bench_pos_y_offset = 0.001 * sc.units.m

    da.coords["pixel_width"] = 0.0075 * sc.units.m
    da.coords["pixel_height"] = 0.0117188 * sc.units.m

    x_offset = 0.028 * sc.units.m
    y_offset = 0.01195 * sc.units.m

    # Now shift pixels positions to get the correct beam center
    da.coords['position'].fields.x += x_offset
    da.coords['position'].fields.y += y_offset

#Direct beam treated separetly 
direct_beam.coords["sample_position"] = sc.vector(value=[0.0, 0.0, 0.0], unit='m')
direct_beam.coords["pixel_width"] = 0.0075 * sc.units.m
direct_beam.coords["pixel_height"] = 0.0117188 * sc.units.m

x_offset = 0.028 * sc.units.m
y_offset = 0.01195 * sc.units.m

# Now shift pixels positions to get the correct beam center
direct_beam.coords['position'].fields.x += x_offset
direct_beam.coords['position'].fields.y += y_offset

## Masking

The next step is to mask noisy and saturated pixels,
as well as a time-of-flight range that contains spurious artifacts from the beamline components.

**Note:** We use programatic masks here and not those stored in xml files.

### Mask bad pixels

We mask the edges of the detector, which are usually noisy.
We also mask the region close to the center of the beam,
so as to not include saturated pixels in our data reduction.

In [ ]:
mask_edges = (
    (sc.abs(sample.coords['position'].fields.x - x_offset) > sc.scalar(0.45, unit='m')) |
    (sc.abs(sample.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    sample.coords['position'].fields.x**2 +
    sample.coords['position'].fields.y**2) < sc.scalar(0.035, unit='m')

for da in ds.values():
    da.masks['edges'] = mask_edges
    da.masks['center'] = mask_center


We can inspect that the coordinate corrections and masking were applied correctly by opening the instrument view.

In [ ]:
#scn.instrument_view(sample.hist(), pixel_size=0.0075)

### Mask Bragg peaks in wavelength

We will now take out the time regions with Bragg peaks from the beam stop and detector window, although in reality the peaks appear only close to the beam stop, and will make little difference to

This could be implemented as masking specific time bins for a specific region in space, but for now we keep it simple.

In [ ]:
#TODO: This needs to be established
wavelength_mask = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[True]),
    coords={
        'wavelength': sc.array(
            dims=['wavelength'], values=[2.21, 2.59], unit='angstrom'
        )
    },
)

In [ ]:
#empty_beam.attrs["monitor_1"].value.hist(tof=200).plot()

### Preprocess monitor

In [ ]:
monitors = {}
for key, da in ds.items():
    #TODO: Check this histogramming!
    monitors[f'{key}-incident'] = da.attrs["monitor_1"].value.hist(tof=200)
    monitors[f'{key}-transmission'] = da.attrs["monitor_2"].value.hist(tof=200)
    monitors[f'{key}-incident'].coords['source_position'] = sc.vector(value=[0.0, 0.0, -9.82], unit='m')
    monitors[f'{key}-transmission'].coords['source_position'] = sc.vector(value=[0.0, 0.0, -25.76], unit='m')
    monitors[f'{key}-incident'].coords['position'] = sc.vector(value=[0.0, 0.0, 0.0], unit='m')
    monitors[f'{key}-transmission'].coords['position'] = sc.vector(value=[0.0, 0.0, 0.0], unit='m')

In [ ]:
monitor_graph = sans.conversions.sans_monitor()

monitors = {
     key: mon.transform_coords('wavelength', graph=monitor_graph)
     for key, mon in monitors.items()
}

In [ ]:
sc.plot(monitors, norm='log', grid=True)

In [ ]:

# Define range outside of which monitor data is considered to be background
non_background_range = sc.array(
    dims=['wavelength'], values=[0.6, 13.7], unit='angstrom'
)

#TODO: It may be required that monitors are histogrammed
# Run preprocessing
monitors = sans.i_of_q.preprocess_monitor_data(
    monitors, non_background_range=non_background_range, wavelength_bins=wavelength_bins
)

# Unpack monitors to make steps below easier
sample_monitors = {
    'incident': monitors['sample_trans-incident'],
    'transmission': monitors['sample_trans-transmission'],
}

empty_beam_monitors = {
    'incident': monitors['empty_beam-incident'],
    'transmission': monitors['empty_beam-transmission'],
}

#Check this! It seems from Mantid code sample_monitors are the same as backgrounds
background_monitors = {
    'incident': monitors['background_trans-incident'],
    'transmission': monitors['background_trans-transmission'],
}

In [ ]:
sc.plot(monitors['empty_beam-transmission'],  grid=True)

### Direct beam midpoints
This operation is needed for performing interpolation. Later, this can probably be dome i_of_q module, however we still need to see what exactly the format of direct beam function would be. 

In [ ]:
direct_beam = direct_beam.rebin(wavelength=wavelength_bins)
direct_beam.coords['wavelength'] = sc.midpoints(direct_beam.coords['wavelength'])

In [ ]:
#Using just single spectrum for testing purposes
#direct_beam = direct_beam['spectrum',45874]

In [ ]:
# data=ds['sample'].copy()
# data_monitors = sample_monitors
# direct_monitors = sample_monitors

# graph = sans.conversions.sans_elastic(gravity=gravity)
# data = data.transform_coords("wavelength", graph=graph)


# if wavelength_mask is not None:
# # If we have binned data and the wavelength coord is multi-dimensional, we need
# # to make a single wavelength bin before we can mask the range.
#     if data.bins is not None:
#         dim = wavelength_mask.dim
#         if (dim in data.bins.coords) and (dim in data.coords):
#             data = data.bin({dim: 1})
#     data = sans.common.mask_range(data, wavelength_mask)
#     data_monitors = {
#             key: sans.common.mask_range(mon, wavelength_mask) for key, mon in data_monitors.items()
#         }
#     direct_monitors = {
#             key: sans.common.mask_range(mon, wavelength_mask)
#             for key, mon in direct_monitors.items()
#         }
    
# data_transmission_monitor=sample_monitors['transmission']
# direct_incident_monitor=empty_beam_monitors['incident']
# direct_transmission_monitor=empty_beam_monitors['transmission']

# denominator1 = (
#     data_transmission_monitor
#     * direct_incident_monitor
#     / direct_transmission_monitor
# )



In [ ]:
#TODO: Testing with with 1.0 value
#direct_beam.values[direct_beam.values == 0] = 1.0

In [ ]:
direct_beam_wav = sans.i_of_q.resample_direct_beam(
        direct_beam=direct_beam, wavelength_bins=wavelength_bins
)

denominator1 = sans.normalization.iofq_denominator(
    data=ds['sample'],
    data_transmission_monitor=sample_monitors['transmission'],
    direct_incident_monitor=empty_beam_monitors['incident'],
    direct_transmission_monitor=empty_beam_monitors['transmission'],
    direct_beam=direct_beam_wav,
)

In [ ]:
pp.superplot(denominator1)

In [ ]:
pp.superplot(direct_beam)

In [ ]:
direct_beam_selected = direct_beam['spectrum', 173162]

In [ ]:
direct_beam_selected.plot()

In [ ]:
# path = '/Users/wojciechpotrzebowski/SCIPP_SPACE/loki_detector_test/2022-06-24_calibrated_nexus_files/'
# direct_beam_2 = loki.io.load_rkh_wav(f'{path}/DirectBeam_20feb_full_v3.dat')
# #direct_beam = scn.load(f'{path}/flat_g4_final_15Jun23_10-17.nxs')

# direct_beam = sans.i_of_q.resample_direct_beam(
#         direct_beam=direct_beam_2, wavelength_bins=wavelength_bins
# )

# denominator2 = sans.normalization.iofq_denominator(
#     data=ds['sample'],
#     data_transmission_monitor=sample_monitors['transmission'],
#     direct_incident_monitor=empty_beam_monitors['incident'],
#     direct_transmission_monitor=empty_beam_monitors['transmission'],
#     direct_beam=direct_beam,
# )

In [ ]:
#NOTE: direct monitors is the same as sample (and backgroud)
sample_q = sans.to_I_of_Q(data=ds['sample'],
    data_monitors=sample_monitors,
    direct_monitors=empty_beam_monitors,
    direct_beam=direct_beam_selected,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_mask=wavelength_mask)

In [ ]:
direct_beam

In [ ]:
background_q = sans.to_I_of_Q(data=ds['background'],
    data_monitors=background_monitors,
    direct_monitors=empty_beam_monitors,
    direct_beam=direct_beam_selected,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_mask=wavelength_mask)

We are now in a position to subtract the background from the sample measurement:

In [ ]:
result = sample_q.bins.sum() - background_q.bins.sum()
result

In [ ]:
result.plot()

<div class="alert alert-info">

**Note**

Instead of `.bins.sum()`,
one could use `sc.histogram()` above to define different `Q` bins compared to the ones defined at the top of the notebook.
This can be done in event mode, see [here](https://scipp.github.io/user-guide/binned-data/computation.html#Subtraction).

There may be performance advantages to first use a coarse `Q` binning when the computing `I(Q)` numerator,
and use finer binning for the final results.

</div>

## Wavelength bands

It is often useful to process the data in a small number (~10) of separate wavelength bands.

This can be achieved by requesting 10 bands from the `to_I_of_Q` workflow via the `wavelength_bands` argument.

In [ ]:
wavelength_bands = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=11,
                               unit='angstrom')

sample_slices = sans.to_I_of_Q(data=ds['sample'],
    data_monitors=sample_monitors,
    direct_monitors=sample_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    wavelength_mask=wavelength_mask)

background_slices = sans.to_I_of_Q(data=ds['background'],
    data_monitors=background_monitors,
    direct_monitors=background_monitors,
    direct_beam=direct_beam,
    wavelength_bins=wavelength_bins,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_bands=wavelength_bands,
    wavelength_mask=wavelength_mask)

result_slices = sample_slices.bins.sum() - background_slices.bins.sum()
result_slices

In [ ]:
collapsed = sc.collapse(result_slices, keep='Q')

fig2, ax2 = plt.subplots(1, 2, figsize=(10, 4))
sc.plot(collapsed, ax=ax2[0])
sc.plot(collapsed, norm='log', ax=ax2[1])


## Adding reduced data to scicat

In [ ]:


# reduced = sample_local.derive()

# reduced.add_local_files('./offspec.ort', base_path='.')
# reduced.contact_email = 'andrew.mccluskey@ess.eu'
# reduced.input_datasets.append(direct_beam_local.pid)
# reduced.name = 'Reduced reflectometry data from QCS sample'
# reduced.used_software = ['scipp-23.01.1']
# reduced.investigator = 'Andrew R. McCluskey'
# reduced.owner_group = sample_local.owner_group

# reduced_up = client.upload_new_dataset_now(reduced)



## References

<div id="manasi2021"></div>

Manasi I., Andalibi M. R., Atri R. S., Hooton J., King S. M., Edler K. J., **2021**,
*Self-assembly of ionic and non-ionic surfactants in type IV cerium nitrate and urea based deep eutectic solvent*,
[J. Chem. Phys. 155, 084902](https://doi.org/10.1063/5.0059238)